In [162]:
#importing modules
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen as uReq
import ssl

#defining lists
desc=[]
urldata=[]
datepub=[]
src=[]
author=[]
headline=[]

#date list for news pull
datelist=['2016/11','2016/12','2017/1','2017/2','2017/3','2017/4','2017/5','2017/6','2017/7','2017/8','2017/9','2017/10','2017/11','2017/12','2018/1','2018/2','2018/3','2018/4','2018/5','2018/6','2018/7']

#news pull from api in json
for d in datelist:
    ssl._create_default_https_context = ssl._create_unverified_context
    url = "http://api.nytimes.com/svc/archive/v1/"+d+".json?api-key=<API KEY HERE>"
    data = json.load(uReq(url))
    
    #extracting data from json file
    for item in (data["response"]["docs"]):
            headline.append(str(item['headline']['main']))
            author.append(str(item['headline']))
            desc.append(str(item['snippet']))
            urldata.append(str(item['web_url']))
            datepub.append(str(item['pub_date'][:10]))

#creating a dataframe with the extracted data            
dataset = pd.DataFrame(
    {'Date': datepub,
     'Description': desc,
     'Headline': headline,
     'URL': urldata,
    })

df= dataset[['Date',
             'Description',
             'Headline', 
             'URL'
            ]
            ]


##print(len(df))

#saving as csv
df.to_csv('outputnytimes.csv')
df_new=df

#splitting the description into world list
df_new['Description'] = df_new['Description'].astype(str).str.split()
#print(len(df_new))
#print(df)


df_new.to_csv('outputnytimesdata.csv')

102842


In [167]:
#sorting by date
dfnew=df.sort_values(by='Date')
dfnew.to_csv('opnytimesdata.csv')

In [168]:
#splitting headline into individual word list
df_new['Headline'] = df_new['Headline'].astype(str).str.split()

In [169]:
#creating a list of unique dates
uniquedates=np.unique(datepub)
#print(uniquedates)
udates = pd.DataFrame(
    {'Date': uniquedates,
    })

udf= udates[['Date'
            ]
            ]

udf.to_csv('udates.csv')

In [170]:
#appending 
fd=[]
i=0
length=len(dfnew)
for ds in uniquedates:
    descrip=[]
    for i in range (i,length):
        if dfnew.iloc[i,0]==ds:
            descrip += dfnew.iloc[i,1]+dfnew.iloc[i,2]
            i+=1
            #print(i)
        else:
            #print("this is last",i)
            break
    #print(ds)
    fd.append(descrip)
    #print(descrip)

dataset1 = pd.DataFrame(
    {'Date': uniquedates,
     'Description': fd,
    })

fdf= dataset1[['Date',
             'Description',
            ]
            ]       
fdf
fdf.to_csv('finaldataset.csv')

In [171]:
train = pd.read_csv('finaldataset.csv')

In [172]:
#splitting into words
train['og_word_count'] = train['Description'].apply(lambda x: len(str(x).split(" ")))
train[['Description','og_word_count']].head()

,Description,og_word_count
0,"['Investigators', 'looking', 'into', 'corrupti...",6152
1,"['When', 'the', 'star', 'of', 'the', 'ABC', 'c...",6270
2,"['The', 'British', 'government', 'must', 'cons...",7642
3,"['The', 'idea', 'that', 'we', 'can', 'solve', ...",7732
4,"['Donald', 'J.', 'Trump', 'encourages', 'his',...",3650


In [173]:
train['Description'] = train['Description'].str.lower()
train['Description'].head()

0    ['investigators', 'looking', 'into', 'corrupti...
1    ['when', 'the', 'star', 'of', 'the', 'abc', 'c...
2    ['the', 'british', 'government', 'must', 'cons...
3    ['the', 'idea', 'that', 'we', 'can', 'solve', ...
4    ['donald', 'j.', 'trump', 'encourages', 'his',...
Name: Description, dtype: object

In [174]:
#removing all the 
train['Description'] = train['Description'].str.replace('[^\w\s]','')
train['Description'].head()

0    investigators looking into corruption within t...
1    when the star of the abc comedy series dr ken ...
2    the british government must consult parliament...
3    the idea that we can solve problems by buildin...
4    donald j trump encourages his supporters to he...
Name: Description, dtype: object

In [175]:
#removing the stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')

train['Description'] = train['Description'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
train['Description'].head()

0    investigators looking corruption within montre...
1    star abc comedy series dr ken takes trip alway...
2    british government must consult parliament pro...
3    idea solve problems building physical barriers...
4    donald j trump encourages supporters heckle jo...
Name: Description, dtype: object

In [183]:
#saving the final dataset
train['word_count_stem'] = train['Description'].apply(lambda x: len(str(x).split(" ")))
train[['Description','word_count_stem']].head()
train[['Date', 'Description']].to_csv('ProjectData.csv',index=False)